<a href="https://colab.research.google.com/github/yoonjaeuk/2023S-Ajou-ML/blob/main/alopecia_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Data Loader
    * code 파일 디렉토리 위치에, 'train_set, test_set'이라는 이름으로 이미지 파일을 저장합니다.
    * alopecia 파일 디렉토리를 기존대로 4개로 유지합니다. (이 상태로 아래 코드에 따라 로딩하면, output이 4개가 됩니다.)
    * 혹은 (이미지 파일을 2개로 합쳐서 output을 2개로 할 수도 있습니다.)

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
trans = transforms.Compose([transforms.Resize((640, 480)), transforms.ToTensor()])

train_dataset = torchvision.datasets.ImageFolder(root="./train_set",transform=trans)
test_dataset= torchvision.datasets.ImageFolder(root="./test_set", transform=trans)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=4)

ModuleNotFoundError: No module named 'torch'

## 2. Net Implementation
    * 자유롭게 구성해볼 수 있겠습니다. (저는 일단 단순하게 구현해봤습니다 -> 현재 정답률이 74% 정도로 나오는 수준으로 구성되어 있습니다)
    * data load에서 tensor size와 호환되도록 구성하셔야 합니다. (혹은 data load를 바꾸셔도 됩니다.)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # [ NET Structure ]

        # input: 320*240*3
        # conv1 with batch nomalization: 640*480*3 -> 636*476*10 (kernel size = 5)
        # pool1: 636*476*10 -> 318*238(*10)
        # conv2 with batch nomalization: 318*238 -> 314*234 (kernel size = 5)
        # pool2: 314*234 -> 157*117
        # conv3 with batch nomalization: 157*117 -> 153*113 (kernel size = 5)
        # pool3: 153*113 -> 76*56
        # conv4 with batch nomalization: 76*56 -> 74*54 (kernel size = 5)
        # pool4: 74*54 -> 37*27
        # conv5 with batch nomalization: 37*27 -> 33*23 (kernel size = 5)
        # pool5: 33*23 -> 16*11
        # flatten: 16*11*10 -> 1760
        # fc6 with batch nomalization: 1760 -> 400
        # fc7 with batch nomalization: 400 -> 80
        # fc8 with batch nomalizatio: 80 -> 20
        # fc9: 20 -> 4

        self.pool = nn.MaxPool2d(2, 2)
        self.conv1 = nn.Conv2d(3, 10, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(10)
        self.conv2 = nn.Conv2d(10, 10, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(10)
        self.conv3 = nn.Conv2d(10, 10, kernel_size=5)
        self.bn3 = nn.BatchNorm2d(10)
        self.conv4 = nn.Conv2d(10, 10, kernel_size=5)
        self.bn4 = nn.BatchNorm2d(10)
        self.conv5 = nn.Conv2d(10, 10, kernel_size=5)
        self.bn5 = nn.BatchNorm2d(10)
        self.fc6 = nn.Linear(1760, 400)
        self.bn6 = nn.BatchNorm1d(400)
        self.fc7 = nn.Linear(400, 80)
        self.bn7 = nn.BatchNorm1d(80)
        self.fc8 = nn.Linear(80, 20)
        self.bn8 = nn.BatchNorm1d(20)
        self.fc9 = nn.Linear(20, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))
        x = torch.nn.Flatten()(x) # flatten all dimensions except batch
        x = F.relu(self.bn6(self.fc6(x)))
        x = F.relu(self.bn7(self.fc7(x)))
        x = F.relu(self.bn8(self.fc8(x)))
        x = F.softmax(self.fc9(x), dim=1)
        return x

net = Net()

## 3. Train and Test
    * mps는 mac에서 gpu를 이용하기 위한 코드입니다. 따라서 위도우 사용자 혹은 코랩 이용자의 경우는 "cuda"로 코드를 바꿔서 진행하시면 gpu를 이용하실 수 있습니다. (HW4 참고)
    * learning rate, optimizer 등은 자유롭게 구성하시면 좋을 것 같습니다.
    * train 과정과 test과정이 실시간으로 드러나도록 구현했고, top1, top2-Accuracy를 출력했습니다.
    * top1은 총 4개의 예측 확률값 중 가장 높은 것이 정답과 일치하는 비율이고, top2는 총 4개의 예측 확률값 중 가장 높은 2개 중에 정답이 속하는 비율입니다.

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
torch.cuda.empty_cache()

True
NVIDIA GeForce RTX 3050


In [ ]:
from numba import cuda
cuda.get_current_device().reset()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("[Devise: %s]" %device)
print("-----------------------------------------------------------------------------")
net = Net()
net = net.to(device)
net = torch.nn.DataParallel(net)

learning_rate = 0.01
# learning_rate = 0.005
epochs = 20
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)
# optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

from tqdm import tqdm

def test(model, testloader):
    top1_correct = 0
    top2_correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        pbar1 = tqdm(test_loader, total=len(test_loader))
        for data, target in pbar1:
            if device == "cuda":
                data = data.to(device)
                target = target.to(device)
            out = net(data)

            _, top1_pred = torch.max(out, 1)
            total += target.size(0)
            top1_correct += (top1_pred == target).sum().item()

            _, top2_pred = torch.topk(out, 2, dim=1)
            top2_correct += sum([1 for i in range(target.size(0)) if target[i] in top2_pred[i]])
            pbar1.set_description('[Test]')

        top1_accuracy = 100.0 * top1_correct / total
        top2_accuracy = 100.0 * top2_correct / total

    print("Top-1 Accuracy: %.9f ([%d/%d])" %(top1_accuracy, top1_correct, total))
    print("Top-2 Accuracy: %.9f ([%d/%d])" %(top2_accuracy, top2_correct, total))


for epoch in range(epochs):
    avg_loss = 0
    pbar2 = tqdm(train_loader, total=len(train_loader))
    for data, target in pbar2:
        if device == "cuda":
            data = data.to(device)
            target = target.to(device)
        pred = net(data)
        optimizer.zero_grad()
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
        pbar2.set_description('[Train Epoch: {:>1}] loss = {:>.9}'.format(epoch + 1, avg_loss))

print("-----------------------------------------------------------------------------")
net.eval()
test(net, test_loader)


[Devise: cuda]
-----------------------------------------------------------------------------


[Train Epoch: 20] loss = 0.865035474: 100%|██████████| 579/579 [01:55<00:00,  5.00it/s]  


-----------------------------------------------------------------------------


[Test]: 100%|██████████| 166/166 [00:31<00:00,  5.35it/s]

Top-1 Accuracy: 72.598335855 ([3839/5288])
Top-2 Accuracy: 85.760211800 ([4535/5288])


## 4. Net structure & #params
    * Net의 구조와 parameter 개수를 표현할 수 있도록 했습니다.
    * 자기가 구성한 입력 size를 summary함수의 input에 넣어주셔야 합니다.

In [ ]:
from torchsummary import summary
net.to("cpu")
summary(net, (3, 640, 480))